In [1]:
%load_ext autoreload
%autoreload 2

# Modelo CNN con generador, embedings de smiles y data-augmentation

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datagen import smiles_dict, smiles_to_seq

2023-02-21 12:35:27.664839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

### smiles_dict

smiles_dict nos da un tokenizador para simplificar el problema. Puede ver como se construyó en la notebook **deep_chem**.
Si al momento de correr el modelo con este diccionario encuentra problemas de key_error, puede agregar los faltantes al diccionario

Mirar dentro de **datagen.py** como se usa este diccionario con la función **smiles_to_seq** para tokenizar. El código es muy sencillo

In [ ]:
print(len(smiles_dict), smiles_dict, sep=" / ")

# Carga de los datos

In [ ]:
df = pd.read_csv(
    'data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv'
)

In [ ]:
max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()
max_sequence_len = len(df['canonical_smiles'].iloc[max_len_idx]) + 20

In [ ]:
df.head()

In [ ]:
X = df['canonical_smiles'].values
y = df['pIC50'].values

# Data augmentation:

https://arxiv.org/pdf/1703.07076.pdf

https://github.com/EBjerrum/molvecgen

https://github.com/Ebjerrum/SMILES-enumeration

En la publicación de arriba se describe una técnica de aumentación de datos para los smiles. Leerla si es de su interes (Opcional)

En el módulo **dataug.py**, tomando como referencia los repositorios arriba citados se implementó la aumentación de datos

In [ ]:
from dataaug import SmilesEnumerator

sme = SmilesEnumerator()

for i in range(10):
    print(sme.randomize_smiles('CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1'))
    

# DataGenerator

Construir un generador al que se le pase al instanciarlo:
- X: smiles (formula química)
- y: pIC50
- batch_size
- max_sequence_len (int): La máxima longitud de las secuencias (para hacer el padding)
- data_augmentation (boolean): si quiero hacer o no data-augmentation. 
- shuffle (boolean)

Guardarlo en el módulo **datagen.py** con el nombre de la clase **DataGenerator**

Notar que el módulo **datagen.py** ya tiene una estructura para completar

### Importamos el módulo y lo probamos

In [ ]:
from datagen import DataGenerator

In [ ]:
dgen = DataGenerator(X, y, max_sequence_len, batch_size=16)

In [ ]:
len(dgen) * dgen.batch_size

In [ ]:
for i, (X_b, y_b) in enumerate(dgen):
    print(f'{i}\r', end='')

# Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=88
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size=0.2, 
    random_state=88
)

In [ ]:
len(X_train), len(X_val), len(X_test)

In [ ]:
X_train

In [ ]:
dgen_train = DataGenerator(X_train, y_train, seq_length=max_sequence_len, batch_size=128, data_augmentation=True)
dgen_val = DataGenerator(X_val, y_val, seq_length=max_sequence_len, batch_size=128, data_augmentation=False)
dgen_test = DataGenerator(X_test, y_test, seq_length=max_sequence_len, batch_size=128, data_augmentation=False)

In [ ]:
for i, (X_b, y_b) in enumerate(dgen_test):
    print(f'{i}\r', end='')

In [ ]:
X_b.shape

# Network Model

In [ ]:
from tensorflow.keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Activation, Embedding, Flatten, Dropout, 
    Concatenate, Input,
    Conv1D, MaxPool1D, GlobalAveragePooling1D
)


In [ ]:
# Métrica
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def calculate_r2(y_true, y_pred, _round=8):
    return round(1 - ((y_true - y_pred.reshape(-1)) ** 2).sum() / ((y_true - y_true.mean()) ** 2).sum(), _round)

In [ ]:
mcp = ModelCheckpoint(
    'models/best_model_{epoch}', 
    save_best_only=True, 
    save_format="h5"
)

In [ ]:
# EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,                  # number of epochs with no improvement (0 means the training 
                                  # is terminated as soon as the performance measure gets worse 
                                  # from one epoch to the next)
    restore_best_weights=True
)

In [ ]:
# Tensorboard
tensorboard = TensorBoard(
    log_dir="logs/fit/model-default",
    histogram_freq=1,
    write_graph=False,
    write_images=False
)

In [ ]:
# Implementar modelo de TextCNN

def text_cnn_1d(sequence_length, vocab_size, embedding_size, filter_sizes, num_filters):
    """ """
    
    # ref: https://analyticsindiamag.com/guide-to-text-classification-using-textcnn/

    max_pool_div = 4  # new
    input_x = Input(shape=(sequence_length,), name='input_x')
    embedding = Embedding(vocab_size + 1, embedding_size, name='embedding')(input_x)
    pooled_outputs = []

    for filter_size in filter_sizes:
    
        conv = Conv1D(
            filters=num_filters, 
            kernel_size=filter_size, 
            activation="relu"
        )(embedding)

        max_p = MaxPool1D(
            pool_size=(sequence_length - filter_size + 1) // max_pool_div
        )(conv)

        pooled_outputs.append(max_p)    

    h_pool = Concatenate(axis=2)(pooled_outputs)
    dense = Flatten()(h_pool)
    dense = Dense(100, activation="relu")(dense)
    dense = Dense(50, activation="relu")(dense)
    dense = Dense(1)(dense) # Salida

    model = Model(input_x, dense)
    
    return model

In [ ]:
# Hiperparámtros de referencia
FILTER_SIZES = (3, 4, 5)
NUM_FILTERS = 128
vocab_size = len(smiles_dict)
embeddings_size = 128

In [ ]:
model = text_cnn_1d(
    max_sequence_len,
    vocab_size,
    embeddings_size,
    FILTER_SIZES,
    NUM_FILTERS
)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=[R2])

In [ ]:
## Tensorboard
from tensorboard import notebook
notebook.list() 

# %tensorboard --logdir logs/fit/
# !tensorboard --logdir logs/fit/ -> Run in your cli (with poetry)

In [ ]:
model_name = "modelTEextCNN"
tensorboard.log_dir = f"logs/fit/model-{model_name}-{dt.now().strftime('%Y%m%d_%H%M')}"

EPOCHS = 500  # La idea es que se detenga por el earlyStopping

history = model.fit(
    dgen_train, 
    epochs=EPOCHS, 
    validation_data=dgen_val,
    callbacks=[
        # mcp,
        early_stopping,
        tensorboard
    ]
)

In [ ]:
# Score

X_test_eval = []
y_t_eval = []

for X_t, y_t in dgen_test:
    X_test_eval = X_test_eval + [list(t) for t in X_t]
    y_t_eval = y_t_eval + list(y_t)
X_test_eval = np.array(X_test_eval)
y_test = np.array(y_t_eval)

X_test_eval.shape, y_test.shape
y_pred = model.predict(X_test_eval)

print(f"R2: {calculate_r2(y_test, y_pred, 3)}; ref value: 0.498")

# Evaluación

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(x=y_test, y=y_pred, scatter_kws={'alpha':0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)
plt.show